In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers.wxgb import wxgbModel
from wideboost.wrappers import wxgb

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'forest_fires',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
# Keep numeric columns
X = []
for s in a[1].keys():
    if a[1][s].dtype != tf.string:
        X.append(a[1][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[0].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[0].numpy()[test_idx]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = np.inf

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['rmse'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['reg:squarederror']),
    'eval_metric':hp.choice('eval_metric',[['rmse']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)



{'alpha': 0, 'btype': 'In', 'colsample_bylevel': 0.821535403236767, 'colsample_bytree': 0.6223649761098446, 'eta': 0.31231861062697797, 'eval_metric': ('rmse',), 'extra_dims': 10, 'gamma': 0, 'lambda': 0.003212769306723843, 'max_depth': 7, 'min_child_weight': 0.006959306444868801, 'objective': 'reg:squarederror', 'subsample': 0.8819985214437864}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[12:35:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you

[90]	train-rmse:910157022727883789761113322676027392.00000	test-rmse:910157022727883789761113322676027392.00000

[91]	train-rmse:2209450724337189426524517445851414528.00000	test-rmse:2209450724337189426524517445851414528.00000

[92]	train-rmse:5363534527887447512656868413714464768.00000	test-rmse:5363534527887447512656868413714464768.00000

[93]	train-rmse:13020562048751034776623016023341137920.00000	test-rmse:13020562048751032415439774588518531072.00000

[94]	train-rmse:31607403338852497568622417660457517056.00000	test-rmse:31607403338852497568622417660457517056.00000

[95]	train-rmse:inf	test-rmse:inf                      

[96]	train-rmse:nan	test-rmse:nan                      

[97]	train-rmse:nan	test-rmse:nan                      

[98]	train-rmse:nan	test-rmse:nan                      

[99]	train-rmse:nan	test-rmse:nan                      

NEW BEST VALUE!                                        
{'alpha': 6.837670163047778e-07, 'btype': 'In', 'colsample_bylevel': 0.97927740281

[76]	train-rmse:nan	test-rmse:nan                                       

[77]	train-rmse:nan	test-rmse:nan                                       

[78]	train-rmse:nan	test-rmse:nan                                       

[79]	train-rmse:nan	test-rmse:nan                                       

[80]	train-rmse:nan	test-rmse:nan                                       

[81]	train-rmse:nan	test-rmse:nan                                       

[82]	train-rmse:nan	test-rmse:nan                                       

[83]	train-rmse:nan	test-rmse:nan                                       

[84]	train-rmse:nan	test-rmse:nan                                       

[85]	train-rmse:nan	test-rmse:nan                                       

[86]	train-rmse:nan	test-rmse:nan                                       

[87]	train-rmse:nan	test-rmse:nan                                       

[88]	train-rmse:nan	test-rmse:nan                                       

[89]	train-rmse:nan	test-rmse:nan     

[71]	train-rmse:72.97686	test-rmse:24.05225                             

[72]	train-rmse:72.96090	test-rmse:24.08881                             

[73]	train-rmse:72.94728	test-rmse:24.10689                             

[74]	train-rmse:72.93652	test-rmse:24.11024                             

[75]	train-rmse:72.92748	test-rmse:24.11383                             

[76]	train-rmse:72.91067	test-rmse:24.14607                             

[77]	train-rmse:72.89962	test-rmse:24.15430                             

[78]	train-rmse:72.88256	test-rmse:24.14838                             

[79]	train-rmse:72.87379	test-rmse:24.13922                             

[80]	train-rmse:72.85376	test-rmse:24.15945                             

[81]	train-rmse:72.84153	test-rmse:24.18173                             

[82]	train-rmse:72.83180	test-rmse:24.19508                             

[83]	train-rmse:72.81883	test-rmse:24.22651                             

[84]	train-rmse:72.80343	test-rmse:24.

[65]	train-rmse:73.33248	test-rmse:23.85051                             

[66]	train-rmse:73.32532	test-rmse:23.84247                             

[67]	train-rmse:73.31655	test-rmse:23.85259                             

[68]	train-rmse:73.30732	test-rmse:23.86211                             

[69]	train-rmse:73.29599	test-rmse:23.86425                             

[70]	train-rmse:73.29562	test-rmse:23.83757                             

[71]	train-rmse:73.28020	test-rmse:23.87943                             

[72]	train-rmse:73.26926	test-rmse:23.88925                             

[73]	train-rmse:73.25411	test-rmse:23.92745                             

[74]	train-rmse:73.25291	test-rmse:23.93444                             

[75]	train-rmse:73.25312	test-rmse:23.87480                             

[76]	train-rmse:73.24076	test-rmse:23.90438                             

[77]	train-rmse:73.23697	test-rmse:23.91131                             

[78]	train-rmse:73.23899	test-rmse:23.

...

[54]	train-rmse:31.21451	test-rmse:29.23777                              

[55]	train-rmse:30.69325	test-rmse:29.44081                              

[56]	train-rmse:30.55125	test-rmse:29.86192                              

[57]	train-rmse:30.06405	test-rmse:29.82262                              

[58]	train-rmse:29.96250	test-rmse:29.96727                              

[59]	train-rmse:29.80875	test-rmse:30.07464                              

[60]	train-rmse:29.68043	test-rmse:29.99645                              

[61]	train-rmse:29.60961	test-rmse:30.02939                              

[62]	train-rmse:29.45507	test-rmse:29.98140                              

[63]	train-rmse:29.26689	test-rmse:30.09373                              

[64]	train-rmse:29.05018	test-rmse:30.10114                              

[65]	train-rmse:28.91309	test-rmse:29.94323                              

[66]	train-rmse:28.63543	test-rmse:30.60977                              

[67]	train-rmse:28.48279	

[48]	train-rmse:70.25029	test-rmse:23.34044                              

[49]	train-rmse:70.19273	test-rmse:23.31339                              

[50]	train-rmse:70.10265	test-rmse:23.31539                              

[51]	train-rmse:70.01468	test-rmse:23.35044                              

[52]	train-rmse:69.94230	test-rmse:23.35947                              

[53]	train-rmse:69.89047	test-rmse:23.36423                              

[54]	train-rmse:69.83535	test-rmse:23.36543                              

[55]	train-rmse:69.74710	test-rmse:23.39540                              

[56]	train-rmse:69.69110	test-rmse:23.37749                              

[57]	train-rmse:69.57240	test-rmse:23.35493                              

[58]	train-rmse:69.44617	test-rmse:23.42411                              

[59]	train-rmse:69.40033	test-rmse:23.42492                              

[60]	train-rmse:69.29915	test-rmse:23.40945                              

[61]	train-rmse:69.23667	

[42]	train-rmse:0.73944	test-rmse:38.81957                               

[43]	train-rmse:0.67694	test-rmse:38.82081                               

[44]	train-rmse:0.69156	test-rmse:38.81372                               

[45]	train-rmse:0.66358	test-rmse:38.81314                               

[46]	train-rmse:0.64087	test-rmse:38.82475                               

[47]	train-rmse:0.62910	test-rmse:38.82028                               

[48]	train-rmse:0.64269	test-rmse:38.81686                               

[49]	train-rmse:0.73300	test-rmse:38.82346                               

[50]	train-rmse:0.71903	test-rmse:38.82799                               

[51]	train-rmse:0.65956	test-rmse:38.83367                               

[52]	train-rmse:0.63701	test-rmse:38.87699                               

[53]	train-rmse:0.63463	test-rmse:38.85922                               

[54]	train-rmse:0.63750	test-rmse:38.86154                               

[55]	train-rmse:0.64323	t

[36]	train-rmse:17.60873	test-rmse:34.66323                              

[37]	train-rmse:17.39012	test-rmse:34.72684                              

[38]	train-rmse:16.97387	test-rmse:34.83894                              

[39]	train-rmse:16.54455	test-rmse:34.90648                              

[40]	train-rmse:16.26615	test-rmse:34.96338                              

[41]	train-rmse:15.96516	test-rmse:34.95674                              

[42]	train-rmse:15.68018	test-rmse:35.03810                              

[43]	train-rmse:15.35323	test-rmse:35.16438                              

[44]	train-rmse:15.08837	test-rmse:35.25536                              

[45]	train-rmse:14.88577	test-rmse:35.30161                              

[46]	train-rmse:14.64588	test-rmse:35.34864                              

[47]	train-rmse:14.42569	test-rmse:35.40659                              

[48]	train-rmse:14.17975	test-rmse:35.49621                              

[49]	train-rmse:13.86066	

[30]	train-rmse:37.46372	test-rmse:26.07380                              

[31]	train-rmse:37.19428	test-rmse:26.01028                              

[32]	train-rmse:36.45848	test-rmse:25.98202                              

[33]	train-rmse:35.35869	test-rmse:25.81485                              

[34]	train-rmse:35.05857	test-rmse:26.07546                              

[35]	train-rmse:34.63632	test-rmse:25.75323                              

[36]	train-rmse:34.24463	test-rmse:25.76749                              

[37]	train-rmse:33.84004	test-rmse:25.75671                              

[38]	train-rmse:33.41400	test-rmse:25.87678                              

[39]	train-rmse:33.06807	test-rmse:25.93466                              

[40]	train-rmse:32.64466	test-rmse:26.08654                              

[41]	train-rmse:32.40130	test-rmse:26.02403                              

[42]	train-rmse:32.23361	test-rmse:26.20485                              

[43]	train-rmse:31.75706	

In [4]:
print(best_val)
print(space_eval(spc, best))

21.966488
{'alpha': 0, 'btype': 'R', 'colsample_bylevel': 0.9635342312561294, 'colsample_bytree': 0.7638343126132746, 'eta': 0.04927994565908231, 'eval_metric': ('rmse',), 'extra_dims': 11, 'gamma': 0.002457517747477575, 'lambda': 1.4534195623953765e-06, 'max_depth': 2, 'min_child_weight': 0.0001642407457843776, 'objective': 'reg:squarederror', 'subsample': 0.5304352135689321}


In [ ]:
# Trial 1
# 25.65

# Trial 2
# 20.069896
# {'alpha': 0.0006045952994834295, 'btype': 'In', 'colsample_bylevel': 0.9082700359288202, 'colsample_bytree': 0.7786798180253729, 'eta': 0.07902762841188583, 'eval_metric': ('rmse',), 'extra_dims': 4, 'gamma': 0, 'lambda': 0.18657888379145574, 'max_depth': 2, 'min_child_weight': 4.562009073519124e-06, 'objective': 'reg:squarederror', 'subsample': 0.615225204551588}

# Trial 3
# 63.662841
# {'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.5725543487460323, 'colsample_bytree': 0.6784342945676656, 'eta': 0.0521165068320424, 'eval_metric': ('rmse',), 'extra_dims': 10, 'gamma': 0.9725741716897448, 'lambda': 3.3644070214573744e-05, 'max_depth': 1, 'min_child_weight': 6.1549376270514415e-06, 'objective': 'reg:squarederror', 'subsample': 0.554000979571982}

# Trial 4
# 64.893481
# {'alpha': 0, 'btype': 'In', 'colsample_bylevel': 0.8978922934153515, 'colsample_bytree': 0.6449490314908373, 'eta': 0.13700371901248387, 'eval_metric': ('rmse',), 'extra_dims': 5, 'gamma': 0.001321402171387794, 'lambda': 0, 'max_depth': 1, 'min_child_weight': 0.09318093430243324, 'objective': 'reg:squarederror', 'subsample': 0.6042862112251647}

# Trial 5
# 21.966488
# {'alpha': 0, 'btype': 'R', 'colsample_bylevel': 0.9635342312561294, 'colsample_bytree': 0.7638343126132746, 'eta': 0.04927994565908231, 'eval_metric': ('rmse',), 'extra_dims': 11, 'gamma': 0.002457517747477575, 'lambda': 1.4534195623953765e-06, 'max_depth': 2, 'min_child_weight': 0.0001642407457843776, 'objective': 'reg:squarederror', 'subsample': 0.5304352135689321}